In [ ]:
# Install Google Chrome (latest stable version)
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get -fy install

# Install dependencies for Chromium and ChromeDriver
!apt install -y unzip

# Download and set up ChromeDriver for the correct Chrome version (135)
!wget -N https://storage.googleapis.com/chrome-for-testing-public/135.0.7049.52/linux64/chromedriver-linux64.zip
!unzip chromedriver-linux64.zip
!mv chromedriver-linux64/chromedriver /usr/local/bin/
!chmod +x /usr/local/bin/chromedriver

# Install Selenium
!pip install selenium

# Clean up (optional)
!rm chromedriver-linux64.zip


--2025-04-15 13:31:04--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 173.194.217.190, 173.194.217.136, 173.194.217.93, ...
Connecting to dl.google.com (dl.google.com)|173.194.217.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115264952 (110M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 109.92M  85.9MB/s    in 1.3s    

2025-04-15 13:31:06 (85.9 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [115264952/115264952]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 126315 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (135.0.7049.84-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Pack

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

folder_path = "/content/drive/My Drive/MLA Project"
os.chdir(folder_path)

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
from bs4 import BeautifulSoup
import csv
import os
import pandas as pd

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.binary_location = "/usr/bin/google-chrome"

service = Service("/usr/local/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)

Obtention of recipe links Code

In [ ]:
# Get all links to each cuisine

url = "https://www.allrecipes.com/cuisine-a-z-6740455"
driver.get(url)

target_section = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#mntl-alphabetical-list_1-0")))

cuisine_elements = target_section.find_elements(By.CSS_SELECTOR, "div > ul > li > a")

cuisine_dict = {
    link.text.strip(): link.get_attribute("href").strip() for link in cuisine_elements if link.text and link.get_attribute("href")
}

print(cuisine_dict)

{'Amish and Mennonite': 'https://www.allrecipes.com/recipes/732/us-recipes/amish-and-mennonite/', 'Argentinian': 'https://www.allrecipes.com/recipes/2432/world-cuisine/latin-american/south-american/argentinian/', 'Australian and New Zealander': 'https://www.allrecipes.com/recipes/228/world-cuisine/australian-and-new-zealander/', 'Austrian': 'https://www.allrecipes.com/recipes/718/world-cuisine/european/austrian/', 'Bangladeshi': 'https://www.allrecipes.com/recipes/16100/world-cuisine/asian/bangladeshi/', 'Belgian': 'https://www.allrecipes.com/recipes/719/world-cuisine/european/belgian/', 'Brazilian': 'https://www.allrecipes.com/recipes/1278/world-cuisine/latin-american/south-american/brazilian/', 'Cajun and Creole': 'https://www.allrecipes.com/recipes/272/us-recipes/cajun-and-creole/', 'Canadian': 'https://www.allrecipes.com/recipes/733/world-cuisine/canadian/', 'Chilean': 'https://www.allrecipes.com/recipes/1277/world-cuisine/latin-american/south-american/chilean/', 'Chinese': 'https:

In [ ]:
# Function to get all links of the dishes in a single cuisine

def get_cuisine_dishes_link(cuisine_url):
    driver.get(cuisine_url)

    target_section = wait.until(EC.presence_of_element_located((By.ID, "mntl-taxonomysc-article-list-group_1-0")))

    dish_elements = target_section.find_elements(By.CSS_SELECTOR, "div > div > a")

    dish_dict = {}

    for link in dish_elements:
        href = link.get_attribute("href")
        if not href:
            continue
        try:
            # Find the dish title span inside the <a>
            title_span = link.find_element(By.CSS_SELECTOR, "span.card__title-text")
            dish_title = title_span.text.strip()
            dish_dict[dish_title] = href.strip()
        except NoSuchElementException:
            continue  # If span not found, skip this entry
    return dish_dict

# # Print the extracted dishes
# for name, url in dish_dict.items():
#     print(f"{name}: {url}")

In [ ]:
all_links = {'Cuisine': [], 'Title': [], 'Link': []}

for cuisine_name, cuisine_url in cuisine_dict.items():
    dishes_dict = get_cuisine_dishes_link(cuisine_url)
    size = len(dishes_dict)
    cuisine = [cuisine_name] * size
    titles = list(dishes_dict.keys())
    links = list(dishes_dict.values())
    all_links['Cuisine'].extend(cuisine)
    all_links['Title'].extend(titles)
    all_links['Link'].extend(links)

links_df = pd.DataFrame(all_links)

In [ ]:
links_df

,Cuisine,Title,Link
0,Amish and Mennonite,Cheesy Amish Breakfast Casserole,https://www.allrecipes.com/recipe/229150/chees...
1,Amish and Mennonite,Best Vinegar Coleslaw,https://www.allrecipes.com/recipe/59318/amish-...
2,Amish and Mennonite,Pennsylvania-Dutch Pickled Beets and Eggs,https://www.allrecipes.com/recipe/13743/pennsy...
3,Amish and Mennonite,Amish Macaroni Salad,https://www.allrecipes.com/recipe/74915/amish-...
4,Amish and Mennonite,Amish Friendship Bread Starter,https://www.allrecipes.com/recipe/7063/amish-f...
...,...,...,...
2264,Vietnamese,Vietnamese Grilled Pork Skewers,https://www.allrecipes.com/recipe/261122/vietn...
2265,Vietnamese,Goi Ga (Vietnamese Chicken and Cabbage Salad),https://www.allrecipes.com/recipe/271155/goi-g...
2266,Vietnamese,Vietnamese Golden Chicken Wings,https://www.allrecipes.com/recipe/142636/vietn...
2267,Vietnamese,Instant Pot Bahn Flan (Vietnamese Flan),https://www.allrecipes.com/recipe/279981/insta...


In [ ]:
# save links_df to csv

links_df.to_csv("recipe_links.csv", index = False)

In [ ]:
# Get all reviews of a specfic dish

# dish_url = "https://www.allrecipes.com/recipe/13199/wonton-soup/"
# driver.get(dish_url)

# target_section = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#recipeUgc1 > div > div:nth-child(4) > div.feedback-list__items")))

# review_elements = target_section.find_elements(By.CSS_SELECTOR, "div.feedback-list__item")

# reviews_df = pd.Dataframe({"User": []})

# for review in review_elements:
#     print(review.find_element(By.CSS_SELECTOR, "div.feedback.review > div > span.feedback__display-name").text.strip())


LivelyCow1858
CalmDate5728
Cyndee
Chef Cosmo
UniqueStew2478
Dalton Draper
Langlay
Deadly Dunja
myoldasskitchen


Obtention of Recipe metadata and Reviews Code

In [ ]:
# Test code for one recipe
# Load and scroll the page
category_url = "https://www.allrecipes.com/recipes/201/meat-and-poultry/chicken/"
driver.get(category_url)

# Scroll to load more recipes
scrolls = 5
for i in range(scrolls):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    print(f"Scrolling {i+1}/{scrolls}")
    time.sleep(2)

# Get updated page source
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Collect all recipe links
recipe_links = []
for a in soup.find_all("a", href=True):
    href = a["href"]
    if "/recipe/" in href and href.endswith("/"):
        recipe_links.append(href)

# Remove duplicates
recipe_links = list(set(recipe_links))

# Show result
print(f"\n Found {len(recipe_links)} recipe links.")
for i, link in enumerate(recipe_links[:10]):
    print(f"[{i+1}] {link}")


Scrolling 1/5
Scrolling 2/5
Scrolling 3/5
Scrolling 4/5
Scrolling 5/5

 Found 4 recipe links.
[1] https://www.allrecipes.com/recipe/258886/poulet-roti-a-lalgerienne-algerian-roast-chicken/
[2] https://www.allrecipes.com/recipe/17453/restaurant-style-chicken-scampi/
[3] https://www.allrecipes.com/recipe/275590/marry-me-chicken/
[4] https://www.allrecipes.com/recipe/258873/tajine-de-poulet-aux-carottes-et-patates-douces-chicken-and-sweet-potato-tagine/


In [ ]:
# CODE TO READ FROM recipes_links CSV
# Define the folder path and file name
file_name = "recipe_links.csv"
# file_name = "allrecipes_reviews_2.csv"
# Construct the full file path using os.path.join
csv_path = os.path.join(folder_path, file_name)
print(f"Looking for CSV at: {csv_path}")

# Read the CSV using pandas
df = pd.read_csv(csv_path)

# Extract the 'Link' column as a list
recipe_links = df["Link"].dropna().tolist()

# Print how many links were loaded
print(f"Loaded {len(recipe_links)} recipe links.")

 # Show the first 5 as a preview
print("\nFirst 5 recipe links:")
print(recipe_links[:5])

Looking for CSV at: /content/drive/Othercomputers/Asus Zenbook 14/Degree/Y3 S2/ML Applications/Project/recipe_links.csv
Loaded 2269 recipe links.

First 5 recipe links:
['https://www.allrecipes.com/recipe/229150/cheesy-amish-breakfast-casserole/', 'https://www.allrecipes.com/recipe/59318/amish-slaw/', 'https://www.allrecipes.com/recipe/13743/pennsylvania-dutch-pickled-beets-and-eggs/', 'https://www.allrecipes.com/recipe/74915/amish-macaroni-salad/', 'https://www.allrecipes.com/recipe/7063/amish-friendship-bread-starter/']


Use this chunk of code if the datasets are not created. Use the next chunk of code if the datasets have already been created with data partially collected, to continue from the point where you left off.

In [ ]:
# Initialize the recipe ID counter
recipe_counter = 1

# To save to CSV
first_recipe = True
first_review = True
fieldnames_recipes = [
    "recipe_id", "recipe", "prep_time", "cook_time", "total_time", "servings",
    "calories", "fat", "carbs", "protein"]

fieldnames_reviews = ["recipe_id", "recipe", "user", "rating", "date", "review"]

# Define paths and filenames separately
# Create the folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

file_name_reviews = "recipe_reviews.csv"
csv_path_reviews = os.path.join(folder_path, file_name_reviews)
file_name_recipes = "recipe_metadata.csv"
csv_path_recipes = os.path.join(folder_path, file_name_recipes)



# All together
for url in recipe_links:
    print(f"Processing: {url}")
    driver.get(url)
    time.sleep(2)

    # Step 1: Scroll down to make sure the 'Load More Reviews' button is in the viewport
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # wait for any dynamic content to load

    # Step 2: Check for "Load More Reviews" button and click it until max_clicks (CHANGE AS NEEDED)
    max_clicks = 300  # OBS: We can set this to any number, it will stop when it reaches the last batch of reviews
    click_count = 0

    try:
        # Loop to keep clicking "Load More Reviews" button until max clicks reached or no more found
        while click_count < max_clicks:
            try:
                # Re-locate the "Load More Reviews" button
                load_more = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, 'feedback-list__load-more-button'))  # Updated class
                )
                # print(click_count, "Clicking 'Load More Reviews' button...")
                driver.execute_script("arguments[0].click();", load_more)
                # Wait for more reviews to load
                time.sleep(2)
                # Scroll after each click to load more content
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                click_count += 1
            except Exception as e:
                # print("No more 'Load More Reviews' button")
                break  # Break the loop when button is no longer clickable

    except Exception as e:
        print("Error:", e)

    # Step 3: Get the full HTML after all reviews loaded
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Recipe ID and name
    recipe_name = soup.find("h1").get_text(strip=True)
    recipe_id = recipe_counter
    recipe_counter += 1
    # print(f"Recipe ID: {recipe_id}")
    # print(f"Recipe Name: {recipe_name}")

    # Extraction after loading soup
    prep_time = cook_time = total_time = servings = nutrition = "N/A"

    # Get metadata based on new class names
    # all_metadata = []
    # Preparation metadata
    metadata_blocks = soup.select("div.mm-recipes-details__content > div")
    for block in metadata_blocks:
      label = block.select_one("div.mm-recipes-details__label")
      value = block.select_one("div.mm-recipes-details__value")

      if label and value:
          label_text = label.get_text(strip=True).lower()
          value_text = value.get_text(strip=True)

          if "prep" in label_text:
              prep_time = value_text
          elif "cook" in label_text:
              cook_time = value_text
          elif "total" in label_text:
              total_time = value_text
          elif "servings" in label_text:
              servings = value_text
    # Nutrition metadata
    nutrition_facts = {}

    # Locate the nutrition facts table
    nutrition_table = soup.select_one("div#mm-recipes-nutrition-facts-summary_1-0 table")

    if nutrition_table:
        for row in nutrition_table.select("tr"):
            cells = row.select("td")
            if len(cells) >= 2:
                # Amount
                amount = cells[0].get_text(strip=True)
                # Calories/Nutrient
                nutrient = cells[1].get_text(strip=True).rstrip(":")
                nutrition_facts[nutrient] = amount
    else:
        print("Nutrition table not found.")

    """print(f"Prep Time: {prep_time}")
    print(f"Cook Time: {cook_time}")
    print(f"Total Time: {total_time}")
    print(f"Servings: {servings}")
    print(f"Nutrition: {nutrition_facts}") """
    recipe_metadata = {
    "recipe_id": recipe_id,
    "recipe": recipe_name,
    "prep_time": prep_time,
    "cook_time": cook_time,
    "total_time": total_time,
    "servings": servings,
    "calories": nutrition_facts.get("Calories", "N/A"),
    "fat": nutrition_facts.get("Fat", "N/A"),
    "carbs": nutrition_facts.get("Carbs", "N/A"),
    "protein": nutrition_facts.get("Protein", "N/A")
}
    # Append the recipe metadata to the list
    # all_metadata.append(recipe_metadata)
    # Write the recipes CSV
    with open(csv_path_recipes, "a", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames_recipes)
        if first_recipe:
            writer.writeheader()
            first_recipe = False
        writer.writerow(recipe_metadata)

    # Step 4: Check review section
    reviews = soup.select("div.review-card, div.feedback-card, div.review")  # try a few known classes

    # reviews = soup.find_all('div', class_='review-card')  # Adjust the class if needed
    # print(f"Found {len(reviews)} reviews")

    all_reviews = []

    for review in reviews:
        # Skip if it's the "Most helpful positive review"
        if review.select_one('.feedback__title') and "Most helpful positive review" in review.select_one('.feedback__title').text:
            continue
        try:
            # Get username
            user_tag = review.select_one('.feedback__display-name-link')
            user = user_tag.get_text(strip=True) if user_tag else "Anonymous"

            # Get review text
            text_tag = review.select_one('.feedback__text')
            text = text_tag.get_text(strip=True) if text_tag else ""

            # Get rating (count how many star icons there are)
            # Count only filled stars, not the outlined ones.
            rating = len(review.select('.feedback__star svg.ugc-icon-star'))

            # Get date
            date_tag = review.select_one('.feedback__meta-date')
            date = date_tag.get_text(strip=True) if date_tag else "Unknown"
            review_data = {
                "recipe_id": recipe_id,
                "recipe": recipe_name,
                "user": user,
                "rating": rating,
                "date": date,
                "review": text
            }
            all_reviews.append(review_data)

        except Exception as e:
            print("Error parsing a review:", e)

    print(f"Extracted {len(all_reviews)} reviews.")

    # Write the CSV
    with open(csv_path_reviews, "a", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames_reviews)
        if first_review:
            writer.writeheader()
            first_review = False
        for review in all_reviews:
            writer.writerow(review)



print(f"CSV recipes saved at: {csv_path_recipes}")
print(f"CSV reviews saved at: {csv_path_reviews}")


Processing: https://www.allrecipes.com/recipe/229150/cheesy-amish-breakfast-casserole/
Extracted 190 reviews.
Processing: https://www.allrecipes.com/recipe/59318/amish-slaw/
Extracted 421 reviews.
Processing: https://www.allrecipes.com/recipe/13743/pennsylvania-dutch-pickled-beets-and-eggs/
Extracted 152 reviews.
Processing: https://www.allrecipes.com/recipe/74915/amish-macaroni-salad/
Extracted 1156 reviews.
Processing: https://www.allrecipes.com/recipe/7063/amish-friendship-bread-starter/
Extracted 287 reviews.
Processing: https://www.allrecipes.com/recipe/74950/my-amish-friends-caramel-corn/
Extracted 1269 reviews.
Processing: https://www.allrecipes.com/recipe/85166/pennsylvania-dutch-corn-pie/
Extracted 60 reviews.
Processing: https://www.allrecipes.com/recipe/140605/amish-peanut-butter/
Extracted 29 reviews.
Processing: https://www.allrecipes.com/recipe/65250/amish-meatloaf/
Extracted 602 reviews.
Processing: https://www.allrecipes.com/recipe/73875/amish-potato-salad/
Extracted 10

In [ ]:
# === Step 1: Read existing recipe names from metadata CSV ===
links_csv = "recipe_links.csv"
links_csv_path = os.path.join(folder_path, links_csv)
links_df = pd.read_csv(links_csv_path)

reviews_csv = "recipe_metadata.csv"
metadata_csv_path = os.path.join(folder_path, reviews_csv)
metadata_df = pd.read_csv(metadata_csv_path)
last_extracted_recipe = metadata_df['recipe'].tolist()[-1]

print("Last extracted recipe: ", last_extracted_recipe)

filtered_links_df = links_df[~links_df['Title'].isin(metadata_df['recipe'])]
recipe_links = filtered_links_df["Link"].dropna().tolist()

recipe_counter = len(metadata_df['recipe']) + 1

# To save to CSV
first_recipe = False
first_review = False

fieldnames_recipes = [
    "recipe_id", "recipe", "prep_time", "cook_time", "total_time", "servings",
    "calories", "fat", "carbs", "protein"]

fieldnames_reviews = ["recipe_id", "recipe", "user", "rating", "date", "review"]

# Define paths and filenames separately
os.makedirs(folder_path, exist_ok = True)
file_name_reviews = "recipe_reviews.csv"
csv_path_reviews = os.path.join(folder_path, file_name_reviews)
file_name_recipes = "recipe_metadata.csv"
csv_path_recipes = os.path.join(folder_path, file_name_recipes)

# All together
for url in recipe_links:
    print(f"Processing: {url}")
    driver.get(url)
    time.sleep(2)

    # Step 1: Scroll down to make sure the 'Load More Reviews' button is in the viewport
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # wait for any dynamic content to load

    # Step 2: Check for "Load More Reviews" button and click it until max_clicks (CHANGE AS NEEDED)
    max_clicks = 300  # OBS: We can set this to any number, it will stop when it reaches the last batch of reviews
    click_count = 0

    try:
        # Loop to keep clicking "Load More Reviews" button until max clicks reached or no more found
        while click_count < max_clicks:
            try:
                # Re-locate the "Load More Reviews" button
                load_more = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, 'feedback-list__load-more-button'))  # Updated class
                )
                # print(click_count, "Clicking 'Load More Reviews' button...")
                driver.execute_script("arguments[0].click();", load_more)
                # Wait for more reviews to load
                time.sleep(2)
                # Scroll after each click to load more content
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                click_count += 1
            except Exception as e:
                # print("No more 'Load More Reviews' button")
                break  # Break the loop when button is no longer clickable

    except Exception as e:
        print("Error:", e)

    # Step 3: Get the full HTML after all reviews loaded
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Recipe ID and name
    recipe_name = soup.find("h1").get_text(strip=True)
    recipe_id = recipe_counter
    recipe_counter += 1
    # print(f"Recipe ID: {recipe_id}")
    # print(f"Recipe Name: {recipe_name}")

    # Extraction after loading soup
    prep_time = cook_time = total_time = servings = nutrition = "N/A"

    # Get metadata based on new class names
    # all_metadata = []
    # Preparation metadata
    metadata_blocks = soup.select("div.mm-recipes-details__content > div")
    for block in metadata_blocks:
      label = block.select_one("div.mm-recipes-details__label")
      value = block.select_one("div.mm-recipes-details__value")

      if label and value:
          label_text = label.get_text(strip=True).lower()
          value_text = value.get_text(strip=True)

          if "prep" in label_text:
              prep_time = value_text
          elif "cook" in label_text:
              cook_time = value_text
          elif "total" in label_text:
              total_time = value_text
          elif "servings" in label_text:
              servings = value_text
    # Nutrition metadata
    nutrition_facts = {}

    # Locate the nutrition facts table
    nutrition_table = soup.select_one("div#mm-recipes-nutrition-facts-summary_1-0 table")

    if nutrition_table:
        for row in nutrition_table.select("tr"):
            cells = row.select("td")
            if len(cells) >= 2:
                # Amount
                amount = cells[0].get_text(strip=True)
                # Calories/Nutrient
                nutrient = cells[1].get_text(strip=True).rstrip(":")
                nutrition_facts[nutrient] = amount
    else:
        print("Nutrition table not found.")

    """print(f"Prep Time: {prep_time}")
    print(f"Cook Time: {cook_time}")
    print(f"Total Time: {total_time}")
    print(f"Servings: {servings}")
    print(f"Nutrition: {nutrition_facts}") """
    recipe_metadata = {
    "recipe_id": recipe_id,
    "recipe": recipe_name,
    "prep_time": prep_time,
    "cook_time": cook_time,
    "total_time": total_time,
    "servings": servings,
    "calories": nutrition_facts.get("Calories", "N/A"),
    "fat": nutrition_facts.get("Fat", "N/A"),
    "carbs": nutrition_facts.get("Carbs", "N/A"),
    "protein": nutrition_facts.get("Protein", "N/A")
}
    # Append the recipe metadata to the list
    # all_metadata.append(recipe_metadata)
    # Write the recipes CSV
    with open(csv_path_recipes, "a", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames_recipes)
        if first_recipe:
            writer.writeheader()
            first_recipe = False
        writer.writerow(recipe_metadata)

    # Step 4: Check review section
    reviews = soup.select("div.review-card, div.feedback-card, div.review")  # try a few known classes

    # reviews = soup.find_all('div', class_='review-card')  # Adjust the class if needed
    # print(f"Found {len(reviews)} reviews")

    all_reviews = []

    for review in reviews:
        # Skip if it's the "Most helpful positive review"
        if review.select_one('.feedback__title') and "Most helpful positive review" in review.select_one('.feedback__title').text:
            continue
        try:
            # Get username
            user_tag = review.select_one('.feedback__display-name-link')
            user = user_tag.get_text(strip=True) if user_tag else "Anonymous"

            # Get review text
            text_tag = review.select_one('.feedback__text')
            text = text_tag.get_text(strip=True) if text_tag else ""

            # Get rating (count how many star icons there are)
            # Count only filled stars, not the outlined ones.
            rating = len(review.select('.feedback__star svg.ugc-icon-star'))

            # Get date
            date_tag = review.select_one('.feedback__meta-date')
            date = date_tag.get_text(strip=True) if date_tag else "Unknown"
            review_data = {
                "recipe_id": recipe_id,
                "recipe": recipe_name,
                "user": user,
                "rating": rating,
                "date": date,
                "review": text
            }
            all_reviews.append(review_data)

        except Exception as e:
            print("Error parsing a review:", e)

    print(f"Extracted {len(all_reviews)} reviews.")

    # Write the CSV
    with open(csv_path_reviews, "a", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames_reviews)
        if first_review:
            writer.writeheader()
            first_review = False
        for review in all_reviews:
            writer.writerow(review)


Last extracted recipe:  Tourtière (French Canadian Meat Pie)
Processing: https://www.allrecipes.com/recipe/22621/old-time-mincemeat-pie/
Extracted 16 reviews.
Processing: https://www.allrecipes.com/recipe/17571/real-french-crepes/
Extracted 105 reviews.
Processing: https://www.allrecipes.com/recipe/9251/pork-pie/
Extracted 28 reviews.
Processing: https://www.allrecipes.com/recipe/98979/lobster-rolls/
Extracted 115 reviews.
Processing: https://www.allrecipes.com/recipe/10049/butter-tarts/
Extracted 186 reviews.
Processing: https://www.allrecipes.com/recipe/33629/cretons/
Extracted 46 reviews.
Processing: https://www.allrecipes.com/recipe/12649/brown-sugar-pie-i/
Extracted 90 reviews.
Processing: https://www.allrecipes.com/recipe/244346/canadian-pork-loin-chops/
Extracted 203 reviews.
Processing: https://www.allrecipes.com/recipe/244351/favorite-fry-bread-tacos/
Extracted 17 reviews.
Processing: https://www.allrecipes.com/recipe/235145/baked-taco-dip/
Extracted 18 reviews.
Processing: ht

KeyboardInterrupt: 